In [15]:
import requests
from bs4 import BeautifulSoup
import csv

In [16]:
resp = requests.get('https://ca.trustpilot.com/review/www.oliverjewellery.ca')
type(resp)
print(dir(resp))

['__attrs__', '__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_content', '_content_consumed', '_next', 'apparent_encoding', 'close', 'connection', 'content', 'cookies', 'elapsed', 'encoding', 'headers', 'history', 'is_permanent_redirect', 'is_redirect', 'iter_content', 'iter_lines', 'json', 'links', 'next', 'ok', 'raise_for_status', 'raw', 'reason', 'request', 'status_code', 'text', 'url']


In [17]:
print(resp.status_code)

200


In [18]:
soup = BeautifulSoup(resp.text, 'html.parser')
print(soup.title)

<title>Oliver Jewellery Reviews | Read Customer Service Reviews of www.oliverjewellery.ca</title>


In [19]:
review_sections = soup.find_all(name='div', attrs={'class': 'styles_cardWrapper__LcCPA styles_show__HUXRb styles_reviewCard__9HxJJ'})
review_sections

[<div class="styles_cardWrapper__LcCPA styles_show__HUXRb styles_reviewCard__9HxJJ"><article class="paper_paper__1PY90 paper_outline__lwsUX card_card__lQWDv card_noPadding__D8PcU styles_reviewCard__hcAvl" data-service-review-card-paper="true"><div class="styles_reviewCardInner__EwDq2"><aside aria-label="Info for Magdalene S" class="styles_consumerInfoWrapper__KP3Ra"><div class="styles_consumerDetailsWrapper__p2wdr"><div class="avatar_imageWrapper__8wdWb" style="width:44px;height:44px;min-width:44px;min-height:44px"><span style="box-sizing:border-box;display:inline-block;overflow:hidden;width:initial;height:initial;background:none;opacity:1;border:0;margin:0;padding:0;position:relative;max-width:100%"><span style="box-sizing:border-box;display:block;width:initial;height:initial;background:none;opacity:1;border:0;margin:0;padding:0;max-width:100%"><img alt="" aria-hidden="true" src="data:image/svg+xml,%3csvg%20xmlns=%27http://www.w3.org/2000/svg%27%20version=%271.1%27%20width=%2744%27%20

In [20]:
resp_url = 'https://ca.trustpilot.com/review/www.oliverjewellery.ca'

reviews_list = []

while len(reviews_list) < 500:
    resp = requests.get(resp_url)
    html = resp.text
    soup = BeautifulSoup(html)

    review_sections = soup.find_all(name='div', attrs={'class': 'styles_cardWrapper__LcCPA styles_show__HUXRb styles_reviewCard__9HxJJ'})

    for section in review_sections:
        try :
            companyName = 'Oliver Jewellery'

            date_element = section.find(name='time', attrs={'datetime': True})
            datePublished = date_element['datetime'] if date_element else None
            
            rating_element = section.find(name='div', attrs={'class': 'styles_reviewHeader__iU9Px'})
            ratingValue = rating_element.attrs['data-service-review-rating'] if rating_element else None
            
            reviewBody_element = section.find(name='p', attrs={'class': "typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn"})
            reviewBody = reviewBody_element.text.strip() if reviewBody_element else None

            reviews_list.append([companyName, datePublished, ratingValue, reviewBody])

        except Exception as e:
            print(f"Error extracting review: {e}")


    # Get the next page URL
    next_page_elem = soup.find('a', attrs={'data-pagination-button-next-link': "true"})
    if next_page_elem:
        resp_url = 'https://ca.trustpilot.com' + next_page_elem['href']
    else:
        print("No more reviews available or blocked by Trustpilot")
        break

reviews_list



[['Oliver Jewellery',
  '2023-09-19T21:41:01.000Z',
  '5',
  'My mother and I ordered quite a few items from the website and we found the ordering process very easy. Packaging was great. Items were as described with their certificates. Very fast delivery. All staff when speaking to them with questions, were very professional and polite. We will definitely be ordering more in the near future.'],
 ['Oliver Jewellery',
  '2023-09-25T18:45:29.000Z',
  '5',
  'Great experience. Jared was wonderful and very patient. I was very happy and I will do business with them again!!JD'],
 ['Oliver Jewellery',
  '2023-09-22T22:34:05.000Z',
  '5',
  'Items arrive really fast and are usually in a better condition than stated on the website.'],
 ['Oliver Jewellery',
  '2023-09-25T16:25:20.000Z',
  '5',
  'I only had about 20 minutes to sneak in and pickup my Anniversary gift. The process was fast and professional! I will be back!'],
 ['Oliver Jewellery',
  '2023-09-15T18:56:57.000Z',
  '5',
  'After shopp

In [21]:
# Write data to CSV file
with open("/Users/xinyuanliang/downloads/reviews_list.csv", 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow(["companyName", "datePublished", "ratingValue", "reviewBody"])
    writer.writerows(reviews_list)